<a href="https://colab.research.google.com/github/meti-94/OpenQA/blob/main/OpenQA_v2/src/linked_extractions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://knowitall.cs.washington.edu/linked_extractions/linked-extractions.zip
!unzip -qq linked-extractions.zip

--2023-11-02 12:01:01--  http://knowitall.cs.washington.edu/linked_extractions/linked-extractions.zip
Resolving knowitall.cs.washington.edu (knowitall.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to knowitall.cs.washington.edu (knowitall.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101939406 (97M) [application/zip]
Saving to: ‘linked-extractions.zip’

linked-extractions. 100%[===================>]  97.22M  12.4MB/s    in 6.6s    

2023-11-02 12:01:08 (14.8 MB/s) - ‘linked-extractions.zip’ saved [101939406/101939406]



In [6]:
!wget http://reverb.cs.washington.edu/reverb_wikipedia_tuples-1.1.txt.gz
!gunzip -qq reverb_wikipedia_tuples-1.1.txt.gz

--2023-11-02 12:04:54--  http://reverb.cs.washington.edu/reverb_wikipedia_tuples-1.1.txt.gz
Resolving reverb.cs.washington.edu (reverb.cs.washington.edu)... 128.208.3.120, 2607:4000:200:12::78
Connecting to reverb.cs.washington.edu (reverb.cs.washington.edu)|128.208.3.120|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28489259 (27M) [application/x-gzip]
Saving to: ‘reverb_wikipedia_tuples-1.1.txt.gz.1’

reverb_wikipedia_tu 100%[===================>]  27.17M  6.04MB/s    in 4.6s    

2023-11-02 12:04:59 (5.86 MB/s) - ‘reverb_wikipedia_tuples-1.1.txt.gz.1’ saved [28489259/28489259]



In [7]:
import pandas as pd
import hashlib
import numpy as np

In [8]:
linking_df = pd.read_csv('/content/linked-extractions.tsv', sep='\t', header=None, index_col=False)
linking_df.columns = ['argument1', 'relation_phrase', 'argument2', 'freebase_ID_argument1', 'freebase_entity_name', 'link_score', 'link_ambiguity_score', 'idk']

In [9]:
reverb_df = pd.read_csv('/content/reverb_wikipedia_tuples-1.1.txt', sep='\t', header=None, index_col=False)
reverb_df.columns = ['ExID', 'arg1', 'rel', 'arg2', 'narg1', 'nrel', 'narg2', 'csents', 'conf', 'urls']

In [10]:
reverb_df['reverb_no'] = reverb_df.index.to_list()
reverb_linked = reverb_df.merge(linking_df, how='left', right_on=['argument1', 'relation_phrase', 'argument2'], left_on=['arg1', 'rel', 'arg2'], sort=False)
reverb_linked['link_score'] = reverb_linked['link_score'].fillna(-1)
# hashlib.sha1(s.encode("utf-8")).hexdigest()
reverb_linked['argument1_uuid'] = reverb_linked.arg1.apply(lambda string:hashlib.sha1(str(string).encode("utf-8")).hexdigest())
reverb_linked['argument2_uuid'] = reverb_linked.arg2.apply(lambda string:hashlib.sha1(str(string).encode("utf-8")).hexdigest())


In [11]:
reverb_linked.to_csv('reverb_linked.csv', index=False)

In [13]:
reverb_linked.head()

,ExID,arg1,rel,arg2,narg1,nrel,narg2,csents,conf,urls,...,argument1,relation_phrase,argument2,freebase_ID_argument1,freebase_entity_name,link_score,link_ambiguity_score,idk,argument1_uuid,argument2_uuid
0,625,$ 10,was deducted for,board,$ 10,be deduct for,board,1,0.95320,http://en.wikipedia.org/wiki/Danbury_and_Norwa...,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,1e20ad13379f617635cc7f83be8cecf555567299,9ea5e97d2f481d5427902c67e55a54ec377e4761
1,705,"$ 10,000",came from,J.J.,"$ 10,000",come from,j.j.,1,0.93859,http://en.wikipedia.org/wiki/Mark_Dann,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,fbfdbf78a3a18fbc6e48ec7fcec1cd91f978ae92,182fb1142d47f2814d7b88789f50a18d9569e93c
2,1007,$ 13.8 million,had been raised for,construction of the memorial,$ 13.8 million,have be raise for,construction of the memorial,1,0.91575,http://en.wikipedia.org/wiki/Pentagon_Memorial...,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,7646408e5966133e148a3f77a75c37a9633cb4b8,c1d6334c2c0222da881bf3d19edf8fe67168567d
3,1112,$ 15.3 million,was paid to,Japan,$ 15.3 million,be pay to,japan,1,0.91996,http://en.wikipedia.org/wiki/Pacific_Proving_G...,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,64deec6d2773ea99c9a09312df0eed0a73b8f98e,fcf29f6cad3232704b33e962ef5194fad3b6817b
4,1179,"$ 16,896",was needed just for,dredging,"$ 16,896",be need just for,dredging,1,0.91575,http://en.wikipedia.org/wiki/Warren_County_Canal,...,NaN,NaN,NaN,NaN,NaN,-1.0,NaN,NaN,1703b3581c4a45a27a519e59e9ec418c9e71e4c0,d6e1948edbbb33c540804f7606a94683249cb012
